In [71]:
from bs4 import BeautifulSoup
import requests
import json

In [72]:
s = requests.Session()

In [73]:
r = s.get('https://www.transtats.bts.gov/Data_Elements.aspx?Data=2')
soup = BeautifulSoup(r.text, "lxml")

In [74]:
def get_options(soup, tag):
    options = []
    get_list = soup.find(id=tag)
    for option in get_list.find_all('option'):
        options.append(option['value'])
    return options

In [75]:
def print_list(label, codes):
    print("\n%s:" % label)
    for code in codes:
        print(code)

In [76]:
def make_request(viewstate, eventvalidation, carrier, airport):
    url = "https://www.transtats.bts.gov/Data_Elements.aspx?Data=2"
    request_elements = (('__EVENTTARGET', ""), 
                        ('__EVENTARGUMENT', ""), 
                        ('__VIEWSTATE', viewstate), 
                        ('__VIEWSTATEGENERATOR', "8E3A4798"), 
                        ('__EVENTVALIDATION', eventvalidation), 
                        ('CarrierList', carrier), 
                        ('AirportList', airport), 
                        ('Submit', "Submit"))

    r = s.post(url, request_elements)
    filename = carrier+"_and_"+airport+".html"
    with open(filename, "w") as f:
        f.write(r.text)

In [77]:
def extract_data(soup):
    data = {"eventvalidation": "",
            "viewstate": ""}
    vs = soup.find(id="__VIEWSTATE")
    viewstate = vs["value"]
    ev = soup.find(id="__EVENTVALIDATION")
    eventvalidation = ev['value']

    return viewstate, eventvalidation

In [78]:
carrier_codes = get_options(soup, 'CarrierList')
airport_codes = get_options(soup, 'AirportList')

In [67]:
viewstate, eventvalidation = extract_data(soup)
make_request(viewstate, eventvalidation, carrier_codes[5], airport_codes[5])